- What is GridFS?
    - BSON has a maximum size of 16MB
    - For anything above, MongoDB will use GridFS to manage large files through sharding

- How RabbitMQ integrates with our architecture?
    - User uploads video to be converted to MP3
    - Request hits out API gateway
    - API gateway stores video in MongoDB, and puts message into queue
        - This lets downstream services know that there is a video to be processed
    - Video to MP3 converter service will consume messages from the queue
        - It gets ID of video from the message
        - Pull video from mongoDB
        - Does the conversion
        - Store mp3 back in mongoDB
        - Put new message in the queue to tell notification service that conversion is done
    - Notification service consumes message from queue, and sends email notification to client
    - Client will then use unique ID form notification / JWT to make request to API gateway to download the mp3 
    - API gateway pulls mp3 from mongoDB and serve to client

    - So the queue is really the center piece of this architecture; it conveys messages between components and ensures that requests and job status messages from the various components are not dropped

- Microservice Jargon
    - Asynchronous/Synchronous Interservice Communication
        - Synchronous communication:
            - This just means that the client sending the request will wait for the response from the server, and does not do anything else in the meantime
            - So the client is "blocked" until it gets a response, hence the term "blocking request"
                - For example, API gateway and auth service is synchronous, because nothing happens until you receive a response, or an error from the auth service
                - This means that the services are tightly coupled
        - Asynchronous communication:
            - Client does not need to await response of downstream service. "Fire and forget"
            - This is achieved by using a queue. The API gateway posts the request to the queue, and continues processing client requests. The queue is responsible for making sure that the request gets sent to the converter
            - If it were synchronous instead, someone converting a big file will block the service for everyone else until the conversion is completed
            - So with the queue, the API gateway and the converter service becomes loosely coupled
    
    - Strong vs Eventual Consistency
        - Imagine we send a synchronous request for conversion. Once the user gets notified that the conversion is done, he is guaranteed to download the latest version of the conversion. Hence, the file transfer is strongly consistent
        - Eventual consistency is, for example, if you do an asynchronous request. And you try to download the mp3 from mongoDB immediately. Though the ID exists, it is null, because conversion is not completed yet. It will eventually be completed, however, user has to contend with a period when the ID exists but the conversion does not

- How does RabbitMQ work?
    - Our use of RabbitMQ in this app is mainly through `channel.basic_publish`
    - Under the hood, posting a message to RabbitMQ comprises
        1. producer sends message to rabbitMQ broker
        2. message enters an "exchange", which basically routes the message to the right queue. 
        3. The message then **binds** to a queue
        4. The message gets popped and consumed by consumer
    - If the queue starts getting stuffed full of messages, we can scale up our consumers to process messages concurrently
    - By default, RabbitMQ will dispatch messages to consumers using round robin

    - You need to make your queue durable, so in the event that the pod dies, the messages will still exist after it spins back up
    ```python
        properties=pika.BasicProperties(delivery_mode=pika.spec.PERSISTENT_DELIVERY_MODE)
    ```

- What is an `Ingress` in K8s?
    - Notice here we have an extra manifest file, where we deploy an Ingress type. What exactly is this?
    - Let's first think about what a service is in K8s
        - A K8s service is a single point of entry to a group of pods
        - for example, here we want a gateway service, which we want to be able to scale up to multiple pods if the requests increase
        - So the service groups all of our gateway pods together, by using a **selector**. In our case, we use a label selector (i.e. `selector: matchLabels: app: gateway`) to determine which pods are part of its group
        - So once we define a service, we can literally forget about individual pods. Everything is abstracted away from us. You just manage the service

    - So this service sits in our cluster, which is a private network
        - However, we need to allow requests from outside our cluster to hit our gateway service. Otherwise obviously there's no way for other people to use your service
        - We do this by using an `Ingress`
        - An Ingress comprises:
            - A loadbalancer
            - A set of rules
                - Rules basically control where requests hitting our load balancer are routed

- To test your service
    - `minikube addons enable ingress`
    - `minikube tunnel` --> Use the loopback address that you specified in etcd